We will work today with 'momentum_data.xlsx' file. It contains several tabs:

- The 1st tab contains the momentum factor as an excess return: $\tilde{r}^{\text{mom}}$.

- The 3rd tab contains returns on portfolios corresponding to scored momentum deciles.

    - $r^{\text{mom(1)}}$ denotes the portfolio of stocks in the lowest momentum decile, the 'losers' with the lowest past returns.
    
    - $r^{\text{mom(10)}}$ denotes the portfolio of stocks in the highest momentum decile.
    
- The 4th tab gives portfolios sorted by momentum and size.

    - $r^{\text{momSU}}$ denotes the portfolio of small stocks in the top 3 deciles of momentum scores.
    
    - $r^{\text{momBD}}$ denotes the portfolio of big-stocks in the bottom 3 deciles of momentum scores.

In [1]:
import pandas as pd
factors = pd.read_excel(r'momentum_data.xlsx', sheet_name='factors (excess returns)', index_col=0)
momentum = pd.read_excel(r'momentum_data.xlsx', sheet_name='momentum (excess returns)', index_col=0)
deciles = pd.read_excel(r'momentum_data.xlsx', sheet_name='deciles (total returns)', index_col=0)
size = pd.read_excel(r'momentum_data.xlsx', sheet_name='size_sorts (total returns)', index_col=0)
risk = pd.read_excel(r'momentum_data.xlsx', sheet_name='risk-free rate', index_col=0)
momentum
deciles

,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
Date,,,,,,,,,,
1927-01-31,-0.0332,-0.0446,0.0269,-0.0033,-0.0041,0.0097,0.0074,0.0036,-0.0041,-0.0024
1927-02-28,0.0753,0.0593,0.0825,0.0727,0.0322,0.0421,0.0283,0.0326,0.0420,0.0701
1927-03-31,-0.0323,-0.0301,-0.0392,-0.0477,-0.0048,-0.0242,0.0205,0.0041,0.0096,0.0547
1927-04-30,0.0204,-0.0320,-0.0247,-0.0137,0.0220,-0.0005,0.0205,-0.0059,0.0159,0.0549
1927-05-31,0.0272,0.0457,0.0595,0.0318,0.0636,0.0580,0.0496,0.0681,0.0811,0.0632
...,...,...,...,...,...,...,...,...,...,...
2022-05-31,-0.0819,-0.0204,0.0337,0.0305,-0.0027,0.0020,-0.0158,-0.0073,-0.0118,0.0154
2022-06-30,-0.1104,-0.1120,-0.1107,-0.1050,-0.0833,-0.0752,-0.0669,-0.0779,-0.0557,-0.1014
2022-07-31,0.1496,0.0957,0.1684,0.0880,0.0796,0.1032,0.0831,0.1162,0.0393,0.0781


### Task 1.

Check that momentum return, $\tilde{r}^{\text{mom:FF}}$, given in the second tab, is constructed:

$\tilde{r}^{\text{mom:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2-
(r^{\text{momBD}}+r^{\text{momSd}})/2$

In [2]:
r_mom_FF = pd.DataFrame(index=momentum.index)
r_mom_FF["UMD 1"] = round(momentum["UMD"], 1)
r_mom_FF["UMD 2"] = round((size["BIG HiPRIOR"] + size["SMALL HiPRIOR"])/2 - (size["BIG LoPRIOR"] + size["SMALL LoPRIOR"])/2, 1)
all(r_mom_FF["UMD 1"] == r_mom_FF["UMD 2"])

True

### Task 2

Fill in next table 1 with the appropriate stats for $\tilde{r}^{\text{mom:FF}}$. Please note, that $\tilde{r}^m$ is the excess market return, MKT; $\tilde{r}^v$ is the value portfolio, HML.

<img src='table1.png'>

In [3]:
mom_perf_over_time = pd.DataFrame(index=["1927-2022", "1927-1993", "1994-2008", "2009-2022"])
mom_perf_over_time.loc["1927-2022", "mean"] = r_mom_FF.loc["1927":"2022", "UMD 1"].mean()
mom_perf_over_time.loc["1927-1993", "mean"] = r_mom_FF.loc["1927":"1993", "UMD 1"].mean()
#буде занадто багато коду (6*4=24 рядки), зробимо функцію

def table_gen(df, r_mom, r_mom_col_name):
    for ind in df.index:
        df.loc[ind, "mean"] = r_mom.loc[ind.split("-")[0]:ind.split("-")[1], r_mom_col_name].mean()
        df.loc[ind, "vol"] = r_mom.loc[ind.split("-")[0]:ind.split("-")[1], r_mom_col_name].std()
        df.loc[ind, "Sharpe"] =  df.loc[ind, "mean"] / df.loc[ind, "vol"]
        df.loc[ind, "skewness"] = r_mom.loc[ind.split("-")[0]:ind.split("-")[1], r_mom_col_name].skew()
        df.loc[ind, "corr. to r^m"] = r_mom.loc[ind.split("-")[0]:ind.split("-")[1], r_mom_col_name].corr(factors.loc[ind.split("-")[0]:ind.split("-")[1], "MKT"])
        df.loc[ind, "corr. to r^v"] = r_mom.loc[ind.split("-")[0]:ind.split("-")[1], r_mom_col_name].corr(factors.loc[ind.split("-")[0]:ind.split("-")[1], "HML"])

table_gen(mom_perf_over_time, r_mom_FF, "UMD 1")
mom_perf_over_time.rename_axis("Subsample", axis=1)

Subsample,mean,vol,Sharpe,skewness,corr. to r^m,corr. to r^v
1927-2022,0.000609,0.049822,0.012228,-2.188826,-0.345801,-0.405482
1927-1993,0.000871,0.047991,0.018142,-3.025223,-0.371853,-0.489162
1994-2008,0.003889,0.060134,0.064671,-0.482249,-0.250653,-0.109297
2009-2022,-0.004242,0.046000,-0.092226,-1.684882,-0.347623,-0.399363


### Task 3

Construct your own long-only implementation:

$\tilde{r}^{\text{momU:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2 - r^f.$

Note that you have to subtract the risk-free rate (RF) to get the excess return of this portfolio.

Fill out Table 2 for the data in the period 1994-2022.

<img src='table2.png'>

In [4]:
r_mom_uFF = pd.DataFrame(index=momentum.loc["1994":"2022"].index)

r_mom_uFF["Long-only"] = (size.loc["1994":"2022", "BIG HiPRIOR"] + size.loc["1994":"2022", "SMALL HiPRIOR"])/2 - risk.loc["1994":"2022", "RF"]
r_mom_uFF["Long-and-short"] = momentum.loc["1994":"2022", "UMD"]

table1 = pd.DataFrame(index=["1994-2022"])
table2 = pd.DataFrame(index=["1994-2022"])
table_gen(table1, r_mom_uFF, "Long-only")
table_gen(table2, r_mom_uFF, "Long-and-short")

res = (pd.concat([table2, table1]))
res.index = ["Long-and-short", "Long-only"]
res.rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr. to r^m,corr. to r^v
Long-and-short,0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601
Long-only,0.009399,0.051645,0.181984,-0.482872,0.903499,-0.154667


### Task 4

Assess how sensitive the threshold for the 'winners' and 'losers' is in the results. Specifically, we compare three constructions:

- long the top 1 decile and short the bottom 1 deciles:

$\tilde{r}^{\text{momD1}} = r^{\text{mom(10)}}-r^{\text{mom(1)}}$

• long the top 3 deciles and short the bottom 3 deciles:

$\tilde{r}^{\text{momD3}} = 
(r^{\text{mom(8)}}+r^{\text{mom(9)}}+r^{\text{mom(10)}})/3-
(r^{\text{mom(1)}}+r^{\text{mom(2)}}+r^{\text{mom(3)}})/3 = 
\frac 13 \sum^{10}_{k=8} r^{\text{mom(k)}} + 
\frac 13 \sum^{3}_{k=1} r^{\text{mom(k)}}$
 

• long the top 5 deciles and short the bottom 5 deciles:

$\tilde{r}^{\text{momD5}} = 
\frac 15 \sum^{10}_{k=6} r^{\text{mom(k)}} + 
\frac 15 \sum^{5}_{k=1} r^{\text{mom(k)}}$

Compare all three constructions by filling out the stats in the table 3 below for the period 1994-2022.

<img src='table3.png'>

In [5]:
d_df = pd.DataFrame()
to_take = deciles["1994":"2022"]
d_df["r_momD1"] = to_take["Hi PRIOR"] - to_take["Lo PRIOR"]
d_df["r_momD3"] = (sum([to_take[k] for k in ("PRIOR 8", "PRIOR 9", "Hi PRIOR")]) - sum([to_take[k] for k in ("Lo PRIOR", "PRIOR 2", "PRIOR 3")]))/3
d_df["r_momD5"] = (sum([to_take[k] for k in ("PRIOR 6", "PRIOR 7", "PRIOR 8", "PRIOR 9", "Hi PRIOR")]) - sum([to_take[k] for k in ("Lo PRIOR", "PRIOR 2", "PRIOR 3", "PRIOR 4", "PRIOR 5")]))/5
first_row, second_row, third_row = pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"])
table_gen(first_row, d_df, "r_momD1")
table_gen(second_row, d_df, "r_momD3")
table_gen(third_row, d_df, "r_momD5")
res = pd.concat([first_row, second_row, third_row])
res.index = ["r_momD1", "r_momD3", "r_momD5"]
res.rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr. to r^m,corr. to r^v
r_momD1,0.006617,0.086011,0.076936,-1.323613,-0.335574,-0.236392
r_momD3,0.002850,0.055590,0.051273,-1.355380,-0.365386,-0.229882
r_momD5,0.001503,0.038690,0.038837,-1.435409,-0.357936,-0.227501


### Task 5
Use the data provided on both small-stock 'winners', $r^{\text{momSU}}$, and small-stock 'losers', $r^{\text{momSD}}$
to construct a small-stock momentum portfolio,

$
r_t^{\text{momS}} = r^{\text{momSU}} - r^{\text{momSD}}
$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$
r_t^{\text{momB}} = r^{\text{momBU}} - r^{\text{momBD}}
$

Fill out Table 4 over the sample 1994-2022.

<img src='table4.png'>

In [6]:
d_df = pd.DataFrame()
d_df["All stocks"] = momentum["UMD"]
d_df["Small stocks"] = size["SMALL HiPRIOR"] - size["SMALL LoPRIOR"]
d_df["Large stocks"] = size["BIG HiPRIOR"] - size["BIG LoPRIOR"]

first_row, second_row, third_row = pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"])
table_gen(first_row, d_df, "All stocks")
table_gen(second_row, d_df, "Small stocks")
table_gen(third_row, d_df, "Large stocks")
res = pd.concat([first_row, second_row, third_row])
res.index = ["All stocks", "Small stocks", "Large stocks"]
res.rename_axis("1994-2022", axis=1)
res.drop(["corr. to r^v"], axis=1).rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr. to r^m
All stocks,0.003946,0.048672,0.081075,-1.433554,-0.306139
Small stocks,0.005228,0.049541,0.105531,-1.750109,-0.306908
Large stocks,0.002660,0.052499,0.050667,-0.857615,-0.277984


### Task 6

Re-do Tables 2 and 4 but for the size portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- Size-sorted portfolios for value. "6 Portfolios Formed on Size and Book-to-Market (2 x 3)"

In [27]:
new_df = pd.read_csv("6_Portfolios_2x3.CSV", index_col=0, nrows=1155, skiprows=15, na_values=[-99.99, -999]).loc["1994":"2023"]

from datetime import datetime
new_df.index = new_df.index.astype("str").map(lambda x: datetime.strptime(x, "%Y%m")).to_period('M').to_timestamp('M')

new_df

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
1994-01-31,1.8803,2.9469,4.8889,2.4154,4.5770,1.6973
1994-02-28,-0.9196,0.9023,-1.0985,-1.7641,-3.0804,-4.6720
1994-03-31,-6.2311,-4.7222,-4.3361,-4.6933,-3.9696,-3.3803
1994-04-30,-0.8882,0.7042,1.4197,0.6158,1.7767,1.6366
1994-05-31,-2.7048,-1.0815,0.1619,1.4041,1.5369,-0.1128
...,...,...,...,...,...,...
2022-05-31,-4.4206,1.6393,4.1352,-2.6261,3.8821,5.6393
2022-06-30,-4.3163,-6.6654,-11.2053,-7.2567,-8.9137,-12.2993
2022-07-31,13.4602,10.7984,8.7106,11.4389,5.1164,7.9956
2022-08-31,0.2912,-2.2551,-2.2318,-4.7173,-2.0541,-1.5841


In [28]:
r_mom6 = pd.DataFrame()
r_mom6["Long-only"] = (new_df["BIG HiBM"] + new_df["SMALL HiBM"])/2 - risk.loc["1994":"2022", "RF"]
r_mom6["Long-and-short"] = (new_df["BIG HiBM"] + new_df["SMALL HiBM"] - new_df["BIG LoBM"] - new_df["SMALL LoBM"])/2
r_mom6

,Long-only,Long-and-short
1994-01-31,3.29060,1.14525
1994-02-28,-2.88735,-1.54340
1994-03-31,-3.86090,1.60400
1994-04-30,1.52545,1.66435
1994-05-31,0.02145,0.67490
...,...,...
2022-05-31,4.88695,8.41060
2022-06-30,-11.75290,-5.96580
2022-07-31,8.35230,-4.09645
2022-08-31,-1.90985,0.30510


In [29]:
first_row, second_row = pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"])
table_gen(first_row, r_mom6, "Long-and-short")
table_gen(second_row, r_mom6, "Long-only")
res = pd.concat([first_row, second_row])
res.index = ["Long-and-short", "Long-only"]
res.rename_axis("1994-2022", axis=1)
#отримали таб.2

1994-2022,mean,vol,Sharpe,skewness,corr. to r^m,corr. to r^v
Long-and-short,0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Long-only,0.998077,5.495839,0.181606,-0.908702,0.866864,0.330352


In [30]:
r_mom62 = pd.DataFrame()
r_mom62["All stocks"] = r_mom6["Long-and-short"]
r_mom62["Small stocks"] = new_df["SMALL HiBM"] - new_df["SMALL LoBM"]
r_mom62["Large stocks"] = new_df["BIG HiBM"] - new_df["BIG LoBM"]
first_row, second_row, third_row = pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"])
table_gen(first_row, r_mom62, "All stocks")
table_gen(second_row, r_mom62, "Small stocks")
table_gen(third_row, r_mom62, "Large stocks")
res = pd.concat([first_row, second_row, third_row])
res.index = ["All stocks", "Small stocks", "Large stocks"]
res.rename_axis("1994-2022", axis=1)
res.drop(["corr. to r^v"], axis=1).rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr. to r^m
All stocks,0.132581,3.315845,0.039984,0.207751,-0.102250
Small stocks,0.306017,3.750472,0.081594,0.271015,-0.269082
Large stocks,-0.040855,3.641729,-0.011219,0.015334,0.090916


### Task 7

Re-do Table 3 for the decile portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- "Portfolios Formed on Book-to-Market"

In [38]:
new_df = pd.read_csv("Portfolios_Formed_on_BE-ME.csv", index_col=0, nrows=1155, skiprows=23, na_values=[-99.99, -999]).iloc[:, 9:].loc["1994":"2023"]

from datetime import datetime
new_df.index = new_df.index.astype("str").map(lambda x: datetime.strptime(x, "%Y%m")).to_period('M').to_timestamp('M')

new_df

,Lo 10,Dec 2,Dec 3,Dec 4,Dec 5,Dec 6,Dec 7,Dec 8,Dec 9,Hi 10
1994-01-31,0.99,3.02,3.73,5.19,4.58,3.38,4.42,0.90,2.76,5.35
1994-02-28,-1.66,-1.62,-1.78,-0.41,-2.78,-2.71,-4.30,-4.32,-4.30,-2.39
1994-03-31,-5.27,-5.11,-3.99,-4.79,-3.59,-4.17,-3.94,-3.56,-2.38,-5.96
1994-04-30,0.65,0.66,0.07,1.16,1.57,2.34,1.63,1.39,1.60,2.13
1994-05-31,0.54,0.23,2.42,0.00,2.62,1.12,0.63,-1.68,1.87,0.43
...,...,...,...,...,...,...,...,...,...,...
2022-05-31,-3.22,-2.59,-1.38,2.46,2.45,3.37,6.99,4.50,6.76,5.94
2022-06-30,-8.01,-6.65,-5.74,-6.57,-8.76,-9.26,-11.19,-11.75,-11.81,-14.49
2022-07-31,14.33,8.09,7.05,5.64,6.07,4.86,6.07,9.27,7.38,6.27
2022-08-31,-5.09,-4.92,-2.99,-3.26,-0.83,-3.75,-0.22,-0.77,-2.82,-1.24


In [40]:
d_df = pd.DataFrame()
d_df["r_momD1"] = new_df["Hi 10"] - new_df["Lo 10"]
d_df["r_momD3"] = (sum([new_df[k] for k in ("Dec 8", "Dec 9", "Hi 10")]) - sum([new_df[k] for k in ("Lo 10", "Dec 2", "Dec 3")]))/3
d_df["r_momD5"] = (sum([new_df[k] for k in ("Dec 6", "Dec 7", "Dec 8", "Dec 9", "Hi 10")]) - sum([new_df[k] for k in ("Lo 10", "Dec 2", "Dec 3", "Dec 4", "Dec 5")]))/5
first_row, second_row, third_row = pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"]), pd.DataFrame(index=["1994-2022"])
table_gen(first_row, d_df, "r_momD1")
table_gen(second_row, d_df, "r_momD3")
table_gen(third_row, d_df, "r_momD5")
res = pd.concat([first_row, second_row, third_row])
res.index = ["r_momD1", "r_momD3", "r_momD5"]
res.rename_axis("1994-2022", axis=1)

1994-2022,mean,vol,Sharpe,skewness,corr. to r^m,corr. to r^v
r_momD1,0.098754,5.106326,0.019339,-0.152904,0.178362,0.758798
r_momD3,0.048029,3.350795,0.014334,-0.289519,0.174831,0.819238
r_momD5,0.001467,2.373432,0.000618,-0.039878,0.158168,0.831363
